In [1]:
# import the libraries
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import EarlyStopping
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import Accuracy, F1Score, Precision, Recall
from keras.optimizers import SGD, Adam, Adagrad, RMSprop, Nadam, AdamW
from keras.src.losses import loss
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
#from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
#import seaborn as sns

import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
TRAIN_DATA_DIR = r"D:\downloadsD\archive\dataset\train"
VAL_DATA_DIR = r"D:\downloadsD\archive\dataset\val"

MODEL_SAVE_DIR = r"C:\Users\anar1\Desktop\CNN INGOLSTADT\PHASE3MODEL"

MODEL_NAME = 'PHASE3_MODEL_PRETRAINED'
TESTING_MODE = False #####


MOMENTUM= 0.9
NESTEROV=True

IMG_SIZE = (224, 168)
NUM_CLASSES = 9
CHANNELS = 3

EPOCHS = 30
LEARNING_RATE = 1e-5
PATIENCE = 15
BATCH = 32
SEED = 42
tf.keras.utils.set_random_seed(SEED)

In [3]:
# target class names
if os.path.isdir(TRAIN_DATA_DIR):
    folder_contents = os.listdir(TRAIN_DATA_DIR)
    print(f"Contents of '{TRAIN_DATA_DIR}':")
    for item in folder_contents:
        print(item)
else:
    print(f"Error: '{TRAIN_DATA_DIR}' is not a valid directory.")

Contents of 'D:\downloadsD\archive\dataset\train':
basement
church
entrance
georgianum
kreuztor
ku
pink
room
wfi


In [4]:
# selection of target folders
selected_classes = [x for x in os.listdir(TRAIN_DATA_DIR)]

# 1) Train split
print(f"Loading Training Data (Color Mode: rgb):")
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    labels='inferred',
    label_mode='int',
    color_mode="rgb",
    batch_size=BATCH,
    image_size=IMG_SIZE,
    seed=SEED
)

#2) Validation split
print("\nLoading Validation Data:")
val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DATA_DIR,
    labels='inferred',
    label_mode='int',
    color_mode="rgb",
    batch_size=BATCH,
    image_size=IMG_SIZE,
    seed=SEED
)

print('\n')
print("Classes of train:", train_ds.class_names)
num_classes = len(train_ds.class_names)

print("Classes of validation:", val_ds.class_names)
num_classes = len(val_ds.class_names)

Loading Training Data (Color Mode: rgb):
Found 6769 files belonging to 9 classes.

Loading Validation Data:
Found 1698 files belonging to 9 classes.


Classes of train: ['basement', 'church', 'entrance', 'georgianum', 'kreuztor', 'ku', 'pink', 'room', 'wfi']
Classes of validation: ['basement', 'church', 'entrance', 'georgianum', 'kreuztor', 'ku', 'pink', 'room', 'wfi']


In [5]:
import json
import os

# 1. Extract the class names (Keras automatically sorts them alphabetically)
class_names = train_ds.class_names
class_mapping = {i: name for i, name in enumerate(class_names)}

# 3. Save it as a JSON artifact in your models directory
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
mapping_path = os.path.join(MODEL_SAVE_DIR, 'class_mapping.json')

with open(mapping_path, 'w') as f:
    json.dump(class_mapping, f, indent=4)

print(f"\nSuccessfully saved class mapping to {mapping_path}")
print("Mapping dictionary:", class_mapping)


Successfully saved class mapping to C:\Users\anar1\Desktop\CNN INGOLSTADT\PHASE3MODEL\class_mapping.json
Mapping dictionary: {0: 'basement', 1: 'church', 2: 'entrance', 3: 'georgianum', 4: 'kreuztor', 5: 'ku', 6: 'pink', 7: 'room', 8: 'wfi'}


In [6]:
# normalization
# IF WE USE MOBILENET WE HAVE TO USE ITS OWN RESCALING WHICH I WILL PUT INTO THE LAYERS
# layers.Rescaling(1./127.5, offset=-1)
# train_ds1 = train_ds.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y))
# val_ds1   = val_ds.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y))

In [7]:
# THE DRY RUN LOGIC (smoke test before the attack)
if TESTING_MODE:
    print("testing mode activated and it helps smoke test")
    train_ds = train_ds.take(2)
    val_ds = val_ds.take(1)

#4) Speed, this is not that important but recommended
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(tf.data.AUTOTUNE)

In [8]:
for images, labels in train_ds.take(1):
    print(images.shape)

total_images = len(train_ds) * BATCH
print(f"Number of total training batches: {len(train_ds)}")
print(f"Number of total training images (approximately): {total_images}")

(32, 224, 168, 3)
Number of total training batches: 212
Number of total training images (approximately): 6784


In [9]:
from tensorflow.keras.applications import MobileNetV2

In [10]:
basemodel = tf.keras.applications.MobileNetV2(input_shape=(224,168,3),
                                              include_top=False,
                                              weights="imagenet")

C:\Users\anar1\AppData\Local\Temp\ipykernel_11764\2309580618.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  basemodel = tf.keras.applications.MobileNetV2(input_shape=(224,168,3),


In [11]:
basemodel.trainable=False #i am seting the weights fixed or freezed so they do not get updated

In [12]:
data_augmentation = keras.Sequential([layers.RandomFlip("horizontal"),
                                      layers.RandomRotation(0.1),
                                      layers.RandomZoom(0.1),
                                      layers.RandomTranslation(0.1,0.1),
                                      layers.RandomContrast(0.1),
                                      ])

In [13]:
model= keras.Sequential([layers.Input(shape=(224,168,3)),
                         data_augmentation,
                         layers.Rescaling(1./127.5,offset=-1),
                         basemodel,
                         layers.GlobalAveragePooling2D(), ## Prof. Voigtlaenders suggestion to use this instead of Flatten()
                         
                         layers.Dense(256,activation="relu", kernel_initializer="he_normal"),
                         layers.BatchNormalization(), # i learned this new, so we normalize the outputs of the layers so that mean =0 and var=1
                         layers.Dropout(0.4), # also from the deep learning class
                         
                         layers.Dense(128,activation="relu",kernel_initializer="he_normal"),
                         layers.BatchNormalization(),
                         layers.Dropout(0.3),
                         
                         layers.Dense(64,activation="relu", kernel_initializer="he_normal"),
                         layers.BatchNormalization(),
                         layers.Dropout(0.2),
                         
                         layers.Dense(32,activation="relu",kernel_initializer="he_normal"),
                         
                         layers.Dense(NUM_CLASSES,activation="softmax")]) #num_class in our class in PHASE 3 is equal to 9 since we have combined multiple subclasses to a unified class, before in phase 1 it was 21 categories

In [14]:
early=EarlyStopping(monitor="val_loss",
                    patience=5,
                    restore_best_weights=True,
                    verbose=1)

In [15]:
# Phase a
basemodel.trainable = False

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=[early]
)

Epoch 1/30
212/212 ━━━━━━━━━━━━━━━━━━━━ 461s 2s/step - accuracy: 0.4797 - loss: 1.6120 - val_accuracy: 0.8852 - val_loss: 0.3418
Epoch 2/30
212/212 ━━━━━━━━━━━━━━━━━━━━ 783s 3s/step - accuracy: 0.9053 - loss: 0.3210 - val_accuracy: 0.9223 - val_loss: 0.2323
Epoch 3/30
212/212 ━━━━━━━━━━━━━━━━━━━━ 695s 3s/step - accuracy: 0.9319 - loss: 0.2136 - val_accuracy: 0.9594 - val_loss: 0.1144
Epoch 4/30
 10/212 ━━━━━━━━━━━━━━━━━━━━ 11:23 3s/step - accuracy: 0.9672 - loss: 0.1033